# Test results

With folder `data/cleaned/preds`, we can obtain the results for the position and error metrics but in an accumulated version and not as an average.

In [39]:
import glob, os, pandas as pd, numpy as np, torch, math
from BronchoTrack import metrics


def compute_metrics(folder):
    aape, aade = [], []
    for pred_path in glob.glob(folder):
        df = pd.read_csv(pred_path, index_col=0)
        acum = df.cumsum()
        aape.append(metrics.EuclideanDistance.euclidean(
                torch.from_numpy(acum.iloc[-1, :3].values), 
                torch.from_numpy(acum.iloc[-1, 6:9].values)
            ).numpy())
        rot_targets = acum.iloc[-1, 9:12].values
        rot_preds = acum.iloc[-1, 3:6].values
        rot_targets = torch.from_numpy(np.where(rot_targets < 2*math.pi, rot_targets, rot_targets/(2*math.pi)))
        rot_preds = torch.from_numpy(np.where(rot_preds < 2*math.pi, rot_preds, rot_preds/(2*math.pi)))
        aade.append(metrics.DirectionError.inverse_cos(rot_preds, rot_targets))
    return "AAPE {} +- {}, AADE {} +- {}".format(np.mean(aape), np.std(aape), np.mean(aade), np.std(aade))

## Example

In [40]:
compute_metrics("data/cleaned/preds/*.csv")

'AAPE 3.274902757112164 +- 1.2923962549741415, AADE 0.031104936900250096 +- 0.21248717133009515'

3.274902757112164 0.031104936900250096


In [11]:
df.iloc[:, :3]

,shift_x,shift_y,shift_z
0,-0.286085,-0.040937,-0.252244
1,-0.058546,-0.642844,-0.112465
2,-0.671197,0.091101,0.228670
3,-0.279952,-0.283915,-0.557042
4,0.452790,-0.582968,-0.226855
5,0.297740,0.043043,-0.361968
6,0.448898,-0.850242,0.187509
7,0.573042,-0.551636,0.277836
8,0.203410,-0.413998,0.033761
9,0.406840,-0.390766,-0.487211


In [18]:
torch.from_numpy(acum.iloc[0, :3].values)

tensor([-0.2861, -0.0409, -0.2522], dtype=torch.float64)